# Data Analysis with Visual Analytics

***Combining analytics with visualization***


## Overview

In this notebook we will continue to explore the Divvy bikes dataset using, cuDF, cuGraph, and cuSpatial to see how these analysis results can easily feed directly into visualization tools like hvplot and Datashader.

### cuDF, cuGraph, cuSpatial, hvplot, and Datashader
- [cuDF](https://docs.rapids.ai/api/cudf/stable/), is a GPU DataFrame library for manipulating data with a pandas-like API.

- [cuGraph](https://docs.rapids.ai/api/cugraph/stable/) is a RAPIDS library for GPU accelerated graph library with functionality like NetworkX.

- [cuSpatial](https://docs.rapids.ai/api/cuspatial/stable/) is a collection of GPU accelerated algorithms for computing geo-spatial measures.

- [hvplot](https://hvplot.holoviz.org/) is a high-level plotting API for the PyData ecosystem built on [HoloViews](http://holoviews.org/).

- [Datashader](https://datashader.org/) is a library for high fidelity server side data rendering.

## Imports

In addition to the libraries mentioned above, we will also make use of libraries [cupy](https://docs.cupy.dev/en/stable/), [NumPy](https://numpy.org/), and [Pandas](https://pandas.pydata.org/) directly.

In [27]:
import cudf
import cugraph
import cupy
import cuspatial

import numpy as np
import pandas as pd

import datashader as ds
import datashader.transfer_functions as tf

import hvplot.cudf
import hvplot.pandas

from pathlib import Path


## Loading Data into cuDF

First let's load the data. In addition to the main Divvy `data.csv` file, we will also load the small `stations.csv` file that we prepared in the first notebook. 

In [28]:
DATA_DIR = Path("../data")

In [29]:
# Note: remember to reparse into datetimes
df = cudf.read_csv(DATA_DIR / "data.csv", parse_dates=('starttime', 'stoptime'))

In [30]:
stations = pd.read_csv(DATA_DIR / "stations.csv")

We will want to continue our investigation into weekday vs weekend patterns, so let's first add a column for that:

In [31]:
df["weekday"] = df['starttime'].dt.weekday

## Trying Analysis with CuSpatial 

Let's take a look at some spatial measures and see if there are any interesting features.

We might start with the first station, and see what the max trip length from it is:

In [32]:
r0 = df.iloc[0]
station_id, origin_lon, origin_lat = r0["from_station_id"], r0["longitude_start"], r0["latitude_start"]

The cuSpatial function `lonlat_to_cartesian` will let us quickly compute the x/y distances for every ending trip location (in Kilometers):

In [33]:
sub_df = df[df["from_station_id"]==station_id[0]]
dist = cuspatial.lonlat_to_cartesian(origin_lon[0], origin_lat[0], sub_df["longitude_end"], sub_df["latitude_end"])

CuPy functions can compute derived values on these GPU dataframes:

In [34]:
# good o' pythagorean theorem
cupy.sqrt(cupy.max(dist.x**2 + dist.y**2))

array(11.37479459)

What if we want to compute all trip distances? We can compute the distances using every station as a starting point:

In [35]:
def trip_dists(df):
    results = []

    for idx, row in stations.iterrows():
        station_id, origin_lon, origin_lat = int(row["station_id"]), row["lon"], row["lat"]
        sub_df = df[df["from_station_id"]==station_id]
        res = cuspatial.lonlat_to_cartesian(origin_lon, origin_lat, sub_df["longitude_end"], sub_df["latitude_end"])
        res["dist"] = cupy.sqrt(res.x**2 + res.y**2)
        results.append(res)
        
    return cudf.concat(results)

In [36]:
all_from_dists = trip_dists(df)

## hvplot of Trip Distances
Now that we have all the distances in a dataframe, we can use hvplot to create a plot:

In [37]:
# bin size is chosen after some trial and error
all_from_dists.hvplot.hist(y="dist", normed=True, bins=20)

:Histogram   [dist]   (dist_frequency)

Clearly most trips are fairly short -usually under 2Km. This makes sense when we remember most trip durations are also less than 15min. 

It might also be interesting follow up and break the distribution of trips down weekday vs weekend:

In [38]:
weekend_trips = df[df["weekday"].isin([5, 6])] # weekend days = 5, 6 
weekday_trips = df[df["weekday"].isin(list(range(5)))]  # weekday days = 0-4

In [39]:
# calculate distances from the previous function
weekend_dists = trip_dists(weekend_trips)
weekday_dists = trip_dists(weekday_trips)

In [40]:
all_combined_dists =  cudf.concat([weekday_dists, weekend_dists])
all_combined_dists.head()

,x,y,dist
0,1.430482,-1.504416,2.075945
1,-0.287137,-2.174111,2.192990
2,-0.030118,0.914889,0.915385
3,0.132174,-0.797706,0.808582
4,-0.378072,-1.381667,1.432460


## hvPlot of Weekend vs Weekday Trip Distance
Plotting these two distributions together we can see the weekday (orange) trips peak more at shorter distances and the weekend distributions (blue) has more, longer trips:

In [41]:
weekend_hist = weekend_dists.hvplot.hist(y="dist", alpha=0.5, bin_range=(0, 10), normed=True, color="blue")
weekday_hist = weekday_dists.hvplot.hist(y="dist", alpha=0.5, bin_range=(0, 10), normed=True, color="orange")
weekend_hist * weekday_hist

:Overlay
   .Histogram.I  :Histogram   [dist]   (dist_frequency)
   .Histogram.II :Histogram   [dist]   (dist_frequency)

While interesting to note, there doesn't appear to be any major revelations using a distance analysis approach. 

## Trying Analysis with cuDF

Let's use CuDF direclty to group and aggregate our data to look for anyting intersting about the flow of trips in and out stations. 

We want to look at the daily net flow of trips at each station, i.e. how many more (or less) trips *started* at a station vs *ended* at a station in a given day.

In order to group by day, we first take the "floor" of each timestamp divided by one day:

In [42]:
one_day = np.datetime64(1, 'D').astype('datetime64[ns]').astype('int64') 

# out
df['from_day'] = df['starttime'].astype('int64') // one_day

# in
df['to_day'] = df['stoptime'].astype('int64') // one_day

Now we can group by the station id and hour for both the departing and arriving cases. We name the columns from the size DataFrame `out` and `in` respectively:

In [43]:
df_out = df.groupby(by=["from_station_id", "from_day"]).size().to_frame('out').reset_index()
df_in = df.groupby(by=["to_station_id", "to_day"]).size().to_frame('in').reset_index()

Let's rename the columns to be the same in both DataFrames:

In [44]:
df_out.rename(columns={"from_station_id": "station_id", "from_day": "day"}, inplace=True)
df_in.rename(columns={"to_station_id": "station_id", "to_day": "day"}, inplace=True)

And reset the index to be the (station id, hour) pair:

In [45]:
df_out = df_out.set_index(["station_id", "day"])
df_in = df_in.set_index(["station_id", "day"])

Now we can join these two DataFrames to compute an `flow = out - in` column:

In [46]:
full_df = df_in.join(df_out, how="outer").fillna(0).reset_index()
full_df["flow"] = full_df["out"] - full_df["in"]

Let's also convert our "day" values back to proper timestamps:

In [47]:
full_df["time"] = (full_df["day"] * one_day).astype('datetime64[ns]')
full_df = full_df[["station_id", "time", "flow"]]

Now we can take a glimpse at the resulting DataFrame which has the net `out-in` trip flow by station per day:
- A `+` positive number means there was an excess of trips *starting out* of the station that day.
- A `-` negative number means an excess of trips *ending in* the station that day.

In [48]:
full_df.head()

,station_id,time,flow
0,2,2015-05-08,1
1,2,2015-05-20,1
2,2,2015-10-20,4
3,2,2015-10-23,1
4,2,2015-10-30,1


We might like to look at the maximal behaviour. What is a high number of excess arrivals or departures at a station? Let's pull out individual timeseries for each station id, and look a the max/min for each station:

In [51]:
flows = []
for i in stations.station_id:
    subdf = full_df[full_df.station_id==i].drop("station_id").set_index("time")
    flows.append((i, subdf.flow.max(), subdf.flow.min()))
flows = pd.DataFrame(flows, columns=["station_id", "max_out", "max_in"])

KeyError: 'One or more values not found in axis'

In [52]:
flows

[]

With this information, we can see what stations had the largest ever excess departures (station 192) or arrivals (station 77):

In [ ]:
flows.iloc[flows.max_out.argmax()]

In [ ]:
flows.iloc[flows.max_in.argmin()]

Knowing about excess arrivals vs departures is probably important for Divvy to be able to manually re-allocate bikes. We could ask what fraction of stations ever have a max of more than 30 excess trips:

In [ ]:
len(flows[flows.max_out > 30])

In [ ]:
len(flows[flows.max_in < -30])

While looking at individual stations or max/mins is useful to get preliminary ideas of patterns, it would be better to see it all at once. 
We would try to look at all of these trip flow series for each station at once, using Datashader. First we need to prepare a new Dataframe that has all the series as columns:

In [25]:
series = []

for i in stations.station_id:
    s = full_df[full_df.station_id==i][["time", "flow"]]
    s.rename(columns={"flow": f"s{i}"}, inplace=True)
    s = s.set_index("time")
    series.append(s)
    
df_wide = cudf.concat(series, axis=1).fillna(0)

The resulting Dataframe has a daily time series for every column, one for each station:

In [ ]:
df_wide

## hvplot of Select Station Flows
It's simple to pull out individual stations for comparison using `hvplot`:

In [ ]:
df_wide.hvplot(y=["s77","s81","s192","s195","s268","s287"], alpha=0.4)


The above plot shows some of the more interesting station patterns - which roughly match the overall seasonal flows. Station 195 appears perpetually over taxed, while something nearby station 77 seems to draw in a lot of bikes. Yet its hard to gleen a pattern without the connection between station's flows. 

Bonus points for anyone who knows what anomaly happened on 6/24/2014 (seriously, we're curious). 


### Try It Now
See if you can plot all the stations in an hvplot (it is possible but takes a while to render): 

In [ ]:
# code here

Lastly, lets take a look at the data with Datashader. First we make a funtion `series_shade` that can take a wide dataframe of timeseries like we have made above, and render *all* of the series at once using Datashader:

In [ ]:
# Details here https://datashader.org/user_guide/index.html
def series_shade(df):
    cols = list(df.columns)
    
    itime = cudf.to_datetime(df.index).astype('int64')
    x_range = (itime[0], itime[-1])
    
    y_range = (df.min().min(), df.max().max())
    
    temp = cudf.DataFrame(df)
    temp["itime"] = itime
    
    # the width is 4x365, leaving one pixel width per day
    cvs = ds.Canvas(plot_height=400, plot_width=1460)
    agg = cvs.line(temp, x="itime", y=cols, agg=ds.count(), axis=1)
    
    print(f"y range: ({y_range[0]}, {y_range[1]})")
    return tf.shade(agg, how='linear', cmap=["purple","red","white"])

## Datashader Line Plot of Total Daily Flows
Now let's pass in-out daily net excess data to get a rough datashder plot:

In [ ]:

series_shade(df_wide)

It's not completely clear what we can see here, but it points to some ideas for future exploration. If you squint it does seem that there is an unbalanced flow out of stations vs into stations.

## Datashader Line Plot of Cumulative Daily Flows
As a last experiment, let's make the same plot, but with *cumulative* excess trips:

In [ ]:
df_cumulative = df_wide.cumsum()

In [ ]:
series_shade(df_cumulative)

This view emphasizes the unbalanced flow and is a bit more interesting. It illustrates the notion that Divvy must be engaging in a lot of continual re-allocation of its bikes to offset these excess out bound trips and that the effort is growing in a roughly linear fashion.

If we knew the marginal costs compared to ridership income, it could prove an interesting data point on when network expansion would become prohibitive. However, without that we need to look elsewhere for analysis. 

### Try It Now
Datashader plots can be wrapped in hvplots, much like bokeh plots. Try wrapping the above examples in order to make them more interactive by using [Datashader's usage guide](https://datashader.org/user_guide/Timeseries.html):

In [ ]:
# code here

## Trying Analysis with cuGraph PageRank

In our previous notebook we were able to find some interesting patterns by converting our dataframe into a graph. Here, we will try the `cugraph.pagerank` algorithm to see if it helps succinctly illustrate patterns for the "most popular" stations.

First, let's see what it looks like to compute PageRank for a single hour of the day, e.g. 5PM, by subsetting the data:

In [ ]:
d17 = df[df["hour"]==17]

Then groupby (from_station_id, to_station_id) and take the group size to get all the unique individual routes between stations that hour, and also the number of trips that took each of those routes:

In [ ]:
g17 = df.groupby(by=["from_station_id", "to_station_id"])
routes17 = g17.size().reset_index()
routes17.head()

Now we can create a `cugraph.Graph` with the from and to station IDs:

In [ ]:
G = cugraph.Graph()
G.from_cudf_edgelist(d17, source='from_station_id', destination='to_station_id')
d17_page = cugraph.pagerank(G)
d17_page.head()

PageRank values are relative, and as such do not matter as much as the ranking it produces for the network of trips. Let's see which stations rank as most important at 5PM (on any day):

In [ ]:
d17_top = d17_page.nlargest(20, "pagerank").to_pandas()
d17_top.head()

## hvplot of 5pm Top PageRank Locations
Plotting these stations we can see that at 5PM the most important stations are nearly all downtown, matching our previous notebook findings about a focused downtown core of total trips:

In [ ]:
d17_page_locs = stations[stations.station_id.isin(d17_top.vertex)]
d17_page_locs.hvplot.points(x='lon', y='lat', alpha=0.7, size=300, geo=True, tiles="OSM").opts(width=600, height=600)

Now that we know applying PageRank seems to produce useful results, let's look at how stations rank by weekdays vs weekends. To get proper rankings, we need to compute it for every individual day of the week:

In [ ]:
results = {}
for w in range(7):
    dfw = df[df["weekday"]==w]
    G = cugraph.Graph()
    G.from_cudf_edgelist(dfw, source='from_station_id', destination='to_station_id')
    df_page = cugraph.pagerank(G).nlargest(20, "pagerank")
    results[w] = set(df_page.to_pandas()["vertex"])

Let's find out what stations were continually highest ranked among all weekdays and weekend days:

In [ ]:
weekday = set.intersection(*[results[i] for i in range(5)]) # days 0-4 are weekdays
weekend = set.intersection(results[5], results[6])  # days 5-6 are the weekend

Listing the stations that are ranked important on weekdays and ranked important on weekends, we find that there is little overlap:


In [ ]:
weekend

In [ ]:
weekday

Finally, we can plot these quickly using hvplot again. Let's add a column to denote weekday / weekend so that we can group by that type:

In [ ]:
r1 = stations[stations.station_id.isin(weekend)]
r1 = r1.assign(type="Weekend")

r2 = stations[stations.station_id.isin(weekday)]
r2 = r2.assign(type="Weekday")

result = pd.concat([r1, r2])

## hvplot of Weekend / Weekday Top PageRank Locations
Looking at the plot, nearly all the important weekday stations are downtown, and on the weekend the important stations are further out in popular districts around downtown:

In [ ]:
result.hvplot.points(x='lon', y='lat', by='type', 
                     alpha=0.7, size=300, geo=True, tiles="OSM").opts(width=700, height=600)

The above map of top PageRanked stations for weekend / weekdays matches very well with the ForceAtlas2 clustered graph and time series cross-filtered visualizations of the previous notebook, but in a much more concise and presentable manner. This is the positive result we were hoping to find with our analysis.

## Summary of Analytics Findings 
When running analytics, its critical to have a solid understanding of the underlying data in order to make correct decisions. We tried several analytical approaches to see if we could glean some meaningful patterns. As is often the case, some worked better than others - but because we did extensive exploratory visualization we now have confidence that the weekend / weekday binned PageRank approach will produce accurate results when used for visualizations in our next notebook.